In [ ]:
%load_ext autoreload
%autoreload 2
import cv2
import h5py
import numpy as np
import os
import scipy.io
import torch as t
import torchvision
from torchvision.transforms import functional as F
from utils.config import opt
from model import FasterRCNNVGG16
from trainer import FasterRCNNTrainer
from data.util import  read_image
from utils.vis_tool import vis_bbox
from utils import array_tool as at
from PIL import Image, ImageDraw
%matplotlib inline

In [ ]:
img = read_image('misc/s15538.jpg')
img = t.from_numpy(img)[None]

image = cv2.imread('misc/s15538.jpg')

# Coordinates for the bounding box
y1, x1, y2, x2 = 88, 488, 250, 578  # Example coordinates

# Draw a rectangle with red color (BGR: 0, 0, 255) and line thickness of 2
cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)

#cv2.circle(image, (x2, y2), radius=3, color=(0, 0, 255), thickness=-1)

cv2.imwrite(f'/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/cropped_imgs/test.jpg', image)

In [ ]:
faster_rcnn = FasterRCNNVGG16()
trainer = FasterRCNNTrainer(faster_rcnn).cuda()

In [ ]:
# Utility function to convert tensor to numpy array
def tensor_to_numpy(img_tensor):
    img_np = at.tonumpy(img_tensor).transpose((1, 2, 0))  # Convert image to numpy and transpose to (H, W, C)
    img_np_2 = (img_np * 255).astype(np.uint8)  # Convert to 8-bit format and ensure values are in 0-255 range
    return img_np_2

# Save the image to a file
def save_image(img_pil, save_path):
    imageRGB = cv2.cvtColor(tensor_to_numpy(img_pil[0]), cv2.COLOR_BGR2RGB)
    img = Image.fromarray(imageRGB)
    img.save(save_path)
    print(f'Saved image with bounding boxes: {save_path}')

You'll need to download pretrained model from [google drive](https://drive.google.com/open?id=1cQ27LIn-Rig4-Uayzy_gH5-cW-NRGVzY) 
# 1. model converted from chainer

In [ ]:
# in this machine the cupy isn't install correctly... 
# so it's a little slow
trainer.load('/home/cy/chainer_best_model_converted_to_pytorch_0.7053.pth')
opt.caffe_pretrain=True # this model was trained from caffe-pretrained model
_bboxes, _labels, _scores = trainer.faster_rcnn.predict(img,visualize=True)
vis_bbox(at.tonumpy(img[0]),
         at.tonumpy(_bboxes[0]),
         at.tonumpy(_labels[0]).reshape(-1),
         at.tonumpy(_scores[0]).reshape(-1))
# it failed to find the dog, but if you set threshold from 0.7 to 0.6, you'll find it

# 2. model trained with torchvision pretrained model

In [ ]:
trainer.load('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/pretrained_weights/fasterrcnn_12211511_0.701052458187_torchvision_pretrain.pth')
opt.caffe_pretrain=False # this model was trained from torchvision-pretrained model
_bboxes, _labels, _scores = trainer.faster_rcnn.predict(img,visualize=True)
# vis_bbox(at.tonumpy(img[0]),
#          at.tonumpy(_bboxes[0]),
#          at.tonumpy(_labels[0]).reshape(-1),
#          at.tonumpy(_scores[0]).reshape(-1))
# it failed to find the dog, but if you set threshold from 0.7 to 0.6, you'll find it

# Debug: Print all detected labels and their scores
# print("Detected labels and scores:")
# for label, score in zip(labels, scores):
#     print(f"Label: {label}, Score: {score}")

# Convert predictions to numpy
bboxes = at.tonumpy(_bboxes[0])
labels = at.tonumpy(_labels[0]).reshape(-1)
scores = at.tonumpy(_scores[0]).reshape(-1)

# # Filter predictions to show only "person" class
person_class_id = 14
person_indices = labels == person_class_id
person_bboxes = bboxes[person_indices]
person_labels = labels[person_indices]
person_scores = scores[person_indices]

# # Visualize only "person" class predictions
vis_bbox(at.tonumpy(img[0]),
         person_bboxes,
         person_labels,
         person_scores)

# Save the image
#save_path = '/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/cropped_imgs/orl_without_bboxes.jpg'  # Replace with your desired path
#normalized_tensor = (img - img.min()) / (img.max() - img.min())
#torchvision.utils.save_image(normalized_tensor, save_path, format='jpeg')

# Load the image
image = cv2.imread('misc/s15538.jpg')

# Define the "person" bounding boxes
bounding_boxes = person_bboxes

# Convert bounding box coordinates to integers
bounding_boxes = [[int(coord) for coord in box] for box in bounding_boxes]

# Draw the bounding boxes on the image
for i, box in enumerate(bounding_boxes):
    x1, y1, x2, y2 = box
    #cv2.rectangle(image, (y1, x1), (y2, x2), (0, 255, 0), 2)  # (0, 255, 0) is the color (green), 2 is the thickness

    # Crop the images within the bounding boxes
    cropped_image = image[x1:x2, y1:y2]
    
    # Save the cropped images
    cv2.imwrite(f'/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/cropped_imgs/cropped_image_{i}.jpg', cropped_image)

    print("{}: {}, {}, {}, {}".format(i, y1, x1, y2, x2))

# 3. model trained with caffe pretrained model

In [ ]:
trainer.load('/home/cy/fasterrcnn_12222105_0.712649824453_caffe_pretrain.pth')
opt.caffe_pretrain=True # this model was trained from caffe-pretrained model
_bboxes, _labels, _scores = trainer.faster_rcnn.predict(img,visualize=True)
vis_bbox(at.tonumpy(img[0]),
         at.tonumpy(_bboxes[0]),
         at.tonumpy(_labels[0]).reshape(-1),
         at.tonumpy(_scores[0]).reshape(-1))

In [ ]:
mat_train = scipy.io.loadmat('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/annotation/test/train_test/Train.mat')

data_train = mat_train['Train']

lst_persons = []
lst_image = []
lst_images = []
lst_coordinates = []

for sample in data_train: 
    #print(sample[0][0][0][0]) # Persons
    id_person = int(str(sample[0][0][0][0]).replace('p', '').replace('[', '').replace("'", '').replace(']', ''))
    lst_persons.append(id_person)
    for sample_2 in sample[0][0][0][2][0]:
        #print(sample_2[0]) # Images of person
        name_image = str(sample_2[0]).replace('[', '').replace("'", '').replace(']', '')
        lst_image.append(name_image)
        #print(sample_2[1]) # Coordinates
        number_coordinates = str(sample_2[1]).replace('[', '').replace("'", '').replace(']', '').replace('  ', ' ')
        number_coordinates = number_coordinates.split(" ")
        lst_coordinates.append(number_coordinates)
    lst_images.append(lst_image)
    lst_image = []

#print(lst_persons)
#print(lst_image)
#print(lst_images)
#print(lst_coordinates)
#total_elements = sum(len(sublist) for sublist in lst_images)
#print(total_elements)

#i = 0
j = 0
k = 0
z = 1
#flag_empty_coordinate = 0
counter_number_persons_with_empty_bounding_boxes = 0

for sample in lst_persons:
    name_person = sample
    for sample_2 in lst_images[j]:
        image = cv2.imread(f'datasets/cuhk_sysu/Image/SSM/{sample_2}')
        name_image_cleaner = sample_2.replace('s', '')
        if lst_coordinates[k][0] != '' and lst_coordinates[k][1] != '' and lst_coordinates[k][2] != '' and lst_coordinates[k][3] != '':
            xmin, ymin, width, height = int(lst_coordinates[k][0]), int(lst_coordinates[k][1]), int(lst_coordinates[k][2]), int(lst_coordinates[k][3])
            y2, x2 = ymin + height, xmin + width
            cropped_image = image[ymin:y2, xmin:x2]
            cv2.imwrite(f'datasets/cuhk_sysu/bounding_box_train/{name_person}_c{z}s1_000000_{name_image_cleaner}', cropped_image)
            z = z + 1
        #else:
            #print(name_person, end=',') # Persons with some empty coordinate in an image (not being considered)
            #flag_empty_coordinate = 1
        k = k + 1
    if z == 1:
        counter_number_persons_with_empty_bounding_boxes = counter_number_persons_with_empty_bounding_boxes + 1
    j = j + 1
    z = 1
    #if flag_empty_coordinate == 1:
        #directory = '/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/bounding_box_train/'
        #for filename in os.listdir(directory):
            #if filename.startswith(str(name_person)):
                #file_path = os.path.join(directory, filename)
                #os.remove(file_path)
        #flag_empty_coordinate = 0
    #else:
        #i = i + 1
    #if i == 751:
        #break

print(counter_number_persons_with_empty_bounding_boxes)

In [ ]:
mat_query = scipy.io.loadmat('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/annotation/test/train_test/TestG100.mat')

data_query = mat_query['TestG100']

lst_persons = []
lst_persons_with_empty_coordinates = []
lst_image = []
lst_coordinates = []

for sample in data_query[0]:
     #print(sample[0][0][0][0][3]]) # Persons
    id_person = int(str(sample[0][0][0][3]).replace('p', '').replace('[', '').replace("'", '').replace(']', ''))
    lst_persons.append(id_person)
    for sample_2 in sample[0][0]:
        #print(sample_2[0]) # Image of persons
        name_image = str(sample_2[0]).replace('[', '').replace("'", '').replace(']', '')
        lst_image.append(name_image)
        #print(sample_2[1]) # Coordinates
        number_coordinates = str(sample_2[1]).replace('[', '').replace("'", '').replace(']', '').replace('  ', ' ')
        number_coordinates = number_coordinates.split(" ")
        lst_coordinates.append(number_coordinates)

#print(lst_persons)
#print(lst_image)
#print(lst_coordinates)

# i = 0
j = 0
# k = 0
# z = 1
# flag_empty_coordinate = 0
counter_number_persons_with_empty_bounding_boxes = 0

for sample in lst_persons:
    name_person = sample
    name_image = lst_image[j]
    image = cv2.imread(f'datasets/cuhk_sysu/Image/SSM/{name_image}')
    name_image_cleaner = name_image.replace('s', '')
    if lst_coordinates[j][0] != '' and lst_coordinates[j][1] != '' and lst_coordinates[j][2] != '' and lst_coordinates[j][3] != '':
        xmin, ymin, width, height = int(lst_coordinates[j][0]), int(lst_coordinates[j][1]), int(lst_coordinates[j][2]), int(lst_coordinates[j][3])
        y2, x2 = ymin + height, xmin + width
        cropped_image = image[ymin:y2, xmin:x2]
        cv2.imwrite(f'datasets/cuhk_sysu/query/{name_person}_c1s1_000000_{name_image_cleaner}', cropped_image)
    #z = z + 1
    #else:
        #print(name_person, end=',') # Persons with some empty coordinate in an image (not being considered)
        #flag_empty_coordinate = 1
    #k = k + 1
    #if z == 1:
    else:
        lst_persons_with_empty_coordinates.append(name_person)
        counter_number_persons_with_empty_bounding_boxes = counter_number_persons_with_empty_bounding_boxes + 1
    j = j + 1
    #z = 1
    #if flag_empty_coordinate == 1:
        #directory = '/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/query/'
        #for filename in os.listdir(directory):
            #if filename.startswith(str(name_person)):
                #file_path = os.path.join(directory, filename)
                #os.remove(file_path)
        #flag_empty_coordinate = 0
    #else:
        #i = i + 1
    #if i == 750:
        #break

lst_persons_test = [item for item in lst_persons if item not in lst_persons_with_empty_coordinates]

print(counter_number_persons_with_empty_bounding_boxes)
print(len(lst_persons_test))

In [ ]:
mat_test = scipy.io.loadmat('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/annotation/test/train_test/TestG100.mat')

data_test = mat_test['TestG100']

lst_persons_with_empty_coordinates = []
dict_person_images = {}
dict_image_coordinates = {}

for sample in data_test[0]:
    id_person = str(sample[0][0][0][3]).replace('p', '').replace('[', '').replace("'", '').replace(']', '')
    dict_person_images[id_person] = []
    for sample_2 in sample[1][0]:
        sample_coordinate = sample_2[1]
        if sample_coordinate.size != 0:
            name_image = str(sample_2[0]).replace('[', '').replace("'", '').replace(']', '')
            dict_image_coordinates[name_image] = []
            dict_person_images[id_person].append(name_image)
            number_coordinates = str(sample_coordinate).replace('[', '').replace("'", '').replace(']', '').replace('  ', ' ')
            number_coordinates = number_coordinates.split(" ")
            dict_image_coordinates[name_image].append(number_coordinates)

#print(dict_person_images)
#print(dict_image_coordinates)
#total_elements = sum(len(sublist) for sublist in lst_images)
#print(total_elements)

#i = 0
#j = 0
#k = 0
z = 2
#flag_empty_coordinate = 0
counter_number_persons_with_empty_bounding_boxes = 0

for sample in lst_persons_test:
    name_person = sample
    for key, value in dict_person_images.items():
        if int(key) == name_person:
            for sample_2 in dict_person_images[key]:
                name_image = sample_2
                image = cv2.imread(f'datasets/cuhk_sysu/Image/SSM/{sample_2}')
                name_image_cleaner = sample_2.replace('s', '')
                for key, value in dict_image_coordinates.items():
                    if key == name_image:
                        if dict_image_coordinates[key][0][0] != '' and dict_image_coordinates[key][0][1] != '' and dict_image_coordinates[key][0][2] != '' and dict_image_coordinates[key][0][3] != '':
                            xmin, ymin, width, height = int(dict_image_coordinates[key][0][0]), int(dict_image_coordinates[key][0][1]), int(dict_image_coordinates[key][0][2]), int(dict_image_coordinates[key][0][3])
                            y2, x2 = ymin + height, xmin + width
                            cropped_image = image[ymin:y2, xmin:x2]
                            cv2.imwrite(f'datasets/cuhk_sysu/bounding_box_test/{name_person}_c{z}s1_000000_{name_image_cleaner}', cropped_image)
                            z = z + 1
            if z == 2:
                counter_number_persons_with_empty_bounding_boxes = counter_number_persons_with_empty_bounding_boxes + 1
                lst_persons_with_empty_coordinates.append(name_person)
    z = 2

directory = '/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/query/'
for sample_3 in lst_persons_with_empty_coordinates:
    name_person = sample_3
    for filename in os.listdir(directory):
        if filename.startswith(str(name_person) + '_'):
            file_path = os.path.join(directory, filename)
            os.remove(file_path)

lst_persons_final = [item for item in lst_persons_test if item not in lst_persons_with_empty_coordinates]

print(counter_number_persons_with_empty_bounding_boxes)
print(len(lst_persons_final))

In [ ]:
pool_mat = scipy.io.loadmat('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/annotation/pool.mat')

pool_data = pool_mat['pool']

lst_pool_images = []

for sample in pool_data:
    name_image = str(sample[0]).replace('[', '').replace("'", '').replace(']', '')
    lst_pool_images.append(name_image)

testg100_mat = scipy.io.loadmat('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/annotation/test/train_test/TestG100.mat')

testg100_data = testg100_mat['TestG100']

lst_testg100_images = []
lst_testg100_persons = []

for sample in testg100_data[0]:
    name_image = str(sample[0][0][0][0]).replace('[', '').replace("'", '').replace(']', '')
    id_person = str(sample[0][0][0][3]).replace('p', '').replace('[', '').replace("'", '').replace(']', '')
    #print(sample)
    lst_testg100_images.append(name_image)
    lst_testg100_persons.append(id_person)
    teste = int(id_person)
    if teste in lst_persons:
        print("ok")

# lst_images = []    

# lst_images = [item for item in lst_pool_images if item not in lst_testg100_images]

# dict_testg100_images = {}
# dict_testg100_coordinates = {}

# for sample in testg100_data[0]:
#     id_person = str(sample[0][0][0][3]).replace('p', '').replace('[', '').replace("'", '').replace(']', '')
#     dict_testg100_images[id_person] = []
#     for sample_2 in sample[1][0]:
#         sample_coordinate = sample_2[1]
#         if sample_coordinate.size != 0:
#             name_image = str(sample_2[0]).replace('[', '').replace("'", '').replace(']', '')
#             dict_testg100_coordinates[name_image] = []
#             dict_testg100_images[id_person].append(name_image)
#             number_coordinates = str(sample_coordinate).replace('[', '').replace("'", '').replace(']', '').replace('  ', ' ')
#             number_coordinates = number_coordinates.split(" ")
#             dict_testg100_coordinates[name_image].append(number_coordinates)
#         break
#     break

#common_elements = [element for element in lst_testg100_persons if element in lst_persons]
#print(common_elements)

#print(lst_pool_images)
#print(lst_testg100_images)
#print(lst_testg100_persons)
#print(lst_images)
#print(dict_testg100_images)
#print(dict_testg100_coordinates)
#total_elements = sum(len(sublist) for sublist in lst_images)
#print(total_elements)

# lst_persons_2 = []

# i = 0
# j = 0
# k = 0
# flag_empty_coordinate = 0

# for sample in lst_images:
#     name_person = sample
#     for key, value in dict_testg100_images.items():
#         if name_person in value:
#             id_person = key
#             lst_persons_2.append(int(id_person))

# for key, value in dict_testg100_images.items():
#     id_person = int(key)
#     #if id_person in lst_persons_2:
#         #print(id_person)

# print(lst_persons[0])
# print(lst_persons_2[0])

# Using list comprehension to find common elements
#common_elements = [element for element in lst_persons if element in lst_persons_2]

#print(common_elements)

#print(lst_persons_2)
            
    

# for sample in lst_persons:
#     name_person = sample
#     for key, value in dict_testg100_imgs.items():
#         if name_person in value:
#             id_person = key
#         image = cv2.imread(f'datasets/cuhk_sysu/Image/SSM/{sample_2}')
#         name_image_cleaner = sample_2.replace('s', '')
#         if lst_coordinates[k][0] != '' and lst_coordinates[k][1] != '' and lst_coordinates[k][2] != '' and lst_coordinates[k][3] != '':
#             xmin, ymin, width, height = int(lst_coordinates[k][0]), int(lst_coordinates[k][1]), int(lst_coordinates[k][2]), int(lst_coordinates[k][3])
#             y2, x2 = ymin + height, xmin + width
#             cropped_image = image[ymin:y2, xmin:x2]
#             cv2.imwrite(f'datasets/cuhk_sysu/bounding_box_test/{name_person}_c1s1_000000_{name_image_cleaner}', cropped_image)
#         else:
#             #print(name_person, end=',') # Persons with some empty coordinate in an image (not being considered)
#             flag_empty_coordinate = 1
#         k = k + 1
#     j = j + 1
#     if flag_empty_coordinate == 1:
#         directory = '/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/bounding_box_test/'
#         for filename in os.listdir(directory):
#             if filename.startswith(str(name_person)):
#                 file_path = os.path.join(directory, filename)
#                 os.remove(file_path)
#         flag_empty_coordinate = 0
#     else:
#         i = i + 1
#     if i == 750:
#         break

# for key, value in dict_testg100_imgs.items():
#     if img_name in value:
#         person_identity = key
#         cv2.imwrite(f'datasets/cuhk_sysu/bounding_box_test/{person_identity}_c1s1_000000_0{img_name_cleaner}', cropped_image)

In [ ]:
# Load MAT file
mat = scipy.io.loadmat('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/annotation/test/train_test/TestG100.mat')

# # Display all variable names contained in the file
#print(mat.keys())

# Print header and version information
#print("Header:", mat['__header__'])
#print("Version:", mat['__version__'])
#print("Globals:", mat['__globals__'])

# Access and print data in 'TestG100'
testg100_data = mat['TestG100']
#print("TestG100 Data:", testg100_data)

# Loop through each sample
for sample in testg100_data[0]:  # Assuming test100g_data[0] to get to the actual data if it's structured with an extra dimension
    sample_img = sample[0][0][0]  # Image
    #for sample_2 in sample_img:
        #sample_img_2 = sample_2
    #sample_img_2 = sample[0][0][0][1]  # Location
    #sample_img_3 = sample[1][0][1][1]  # Location
    #print(str(sample_img) + str(sample_img_2) + str(sample_img_3))
    
    #print(str(sample_img))
    #print(str(sample_img[3]))
    #lst_person.append(str(sample_img[3]))
    #i = i + 1

    img_name = str(sample_img[0]).replace('[', '').replace("'", '').replace(']', '')

    img_name_cleaner = img_name.replace('s', '')

    person_identity = str(sample_img[3]).replace('p', '').replace('[', '').replace("'", '').replace(']', '')

    image = cv2.imread(f'datasets/cuhk_sysu/Image/SSM/{img_name}')

    img_coordinates = str(sample_img[1]).replace('[', '').replace(']', '')

    lst_img_coordinates = img_coordinates.split()

    lst_img_coordinates = [int(coordinate) for coordinate in lst_img_coordinates]
    
    xmin, ymin, width, height = lst_img_coordinates[0], lst_img_coordinates[1], lst_img_coordinates[2], lst_img_coordinates[3]
    y2, x2 = ymin + height, xmin + width
    
    cropped_image = image[ymin:y2, xmin:x2]

    cv2.imwrite(f'datasets/cuhk_sysu/query/{person_identity}_c1s1_000000_{img_name_cleaner}', cropped_image)

    #print(str(sample_img[0]))
    #print(str(sample_img[1]))
    #print("{}, {}".format(str(sample_img[0]), str(sample_img[1])))

#lst_person_cleaned = [s.replace('[', '').replace(']', '').replace("'", '').replace('"', '') for s in lst_person]

#unique_set = set(lst_person_cleaned)

# Check for duplicates
#if len(unique_set) != len(lst_person_cleaned):
    #print("There are duplicate elements.")
#else:
    #print("All elements are unique.")

#print(sorted(lst_person_cleaned))
#print(i)

In [ ]:
# Load MAT file
mat = scipy.io.loadmat('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/annotation/pool.mat')

# # Display all variable names contained in the file
#print(mat.keys())

# Print header and version information
#print("Header:", mat['__header__'])
#print("Version:", mat['__version__'])
#print("Globals:", mat['__globals__'])

# Access and print data in 'TestG50'
testg100_data = mat['pool']
#print("TestG100 Data:", testg100_data)

# Loop through each sample
for sample in testg100_data:  # Assuming testg100g_data[0] to get to the actual data if it's structured with an extra dimension
    #sample_img = sample[0][0][0]  # Image
    sample_img = sample  # Image
    #print(str(sample_img[0]))

In [ ]:
# Load pool MAT file
pool_mat = scipy.io.loadmat('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/annotation/pool.mat')

pool_data = pool_mat['pool']

lst_pool_imgs = []

for sample in pool_data:
    sample_img = sample[0]
    img_name = str(sample_img).replace('[', '').replace("'", '').replace(']', '')
    lst_pool_imgs.append(img_name)

# Load TestG100 MAT file
testg100_mat = scipy.io.loadmat('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/annotation/test/train_test/TestG100.mat')

testg100_data = testg100_mat['TestG100']

lst_testg100_imgs = []
lst_testg100_persons = []

for sample in testg100_data[0]:
    sample_img = sample[0][0][0]
    sample_person = sample[0][0]
    img_name = str(sample_img[0]).replace('[', '').replace("'", '').replace(']', '')
    person_identity = str(sample_img[3]).replace('p', '').replace('[', '').replace("'", '').replace(']', '')
    lst_testg100_imgs.append(img_name)
    lst_testg100_persons.append(person_identity)
    

lst_imgs = [item for item in lst_pool_imgs if item not in lst_testg100_imgs]

dict_testg100_imgs = {}

for sample in testg100_data[0]:
    sample_img = sample[0][0][0]
    person_identity = str(sample_img[3]).replace('p', '').replace('[', '').replace("'", '').replace(']', '')
    dict_testg100_imgs[person_identity] = []
    for sample_2 in sample[1][0]:
        sample_coordinate = sample_2[1]
        if sample_coordinate.size != 0:
            sample_img_2 = sample_2[0]
            img_name = str(sample_img_2).replace('[', '').replace("'", '').replace(']', '')
            dict_testg100_imgs[person_identity].append(img_name)

#dict_testg100_imgs = {key: ', '.join(value) for key, value in dict_testg100_imgs.items()}

#print(len(lst_imgs))
#print(lst_testg100_imgs[0])
#print(lst_testg100_persons[0])
#print(lst_imgs[0])
#print(dict_testg100_imgs)
#first_key = next(iter(dict_testg100_imgs))
#first_elements = dict_testg100_imgs[first_key]
#print(f"First key: {first_key}")
#print("Elements:", first_elements)

#def print_duplicates(input_list):
    #i = 0
    #seen = set()
    #duplicates = set()
    #for item in input_list:
        #if item in seen:
            #duplicates.add(item)
        #seen.add(item)
    #for duplicate in duplicates:
        #print(duplicate)
        #i = i + 1
    #print(i)

# Example usage:
#print_duplicates(lst_testg100_imgs)
#print(len(lst_imgs) - 167)

# # Display all variable names contained in the file
#print(mat.keys())

# Print header and version information
#print("Header:", mat['__header__'])
#print("Version:", mat['__version__'])
#print("Globals:", mat['__globals__'])

# Access and print data in 'TestG50'
#testg100_data = mat['TestG100']
#print("TestG100 Data:", testg100_data)

trainer.load('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/pretrained_weights/fasterrcnn_12211511_0.701052458187_torchvision_pretrain.pth')
opt.caffe_pretrain=False # this model was trained from torchvision-pretrained model

#directory = 'datasets/cuhk_sysu/Image/SSM/'
#lst_imgs_path = []

#for root, dirs, files in os.walk(directory):
    #for file in files:
        #img_path = os.path.join(root, file)
        #lst_imgs_path.append(img_path)

#lst_imgs_path = [path.split('SSM/')[1] for path in lst_imgs_path]

# Loop through each sample
#for sample in testg100_data[0]:  # Assuming testg100g_data[0] to get to the actual data if it's structured with an extra dimension
    #sample_img = sample[0][0][0]  # Image
for sample in lst_imgs:
    #sample_img = sample  # Image
    #for sample_2 in sample_img:
        #sample_img_2 = sample_2
    #sample_img_2 = sample[0][0][0][1]  # Location
    #sample_img_3 = sample[1][0][1][1]  # Location
    #print(str(sample_img) + str(sample_img_2) + str(sample_img_3))
    
    #print(str(sample_img))
    #print(str(sample_img[3]))
    #lst_person.append(str(sample_img[3]))
    #i = i + 1

    #img_name = str(sample_img[1][0][1][0]).replace('[', '').replace("'", '').replace(']', '')

    #img_name = str(sample_img[0]).replace('[', '').replace("'", '').replace(']', '')

    #if img_name in lst_imgs_path:
       #print(len(lst_imgs_path))
        
        #pass
    
    #else:

        #print("b")

    img_name = sample

    img_name_cleaner = img_name.replace('s', '')

    img = read_image(f'datasets/cuhk_sysu/Image/SSM/{img_name}')
    
    image = cv2.imread(f'datasets/cuhk_sysu/Image/SSM/{img_name}')
    
    img = t.from_numpy(img)[None]
    
    _bboxes, _labels, _scores = trainer.faster_rcnn.predict(img,visualize=True)
    
    bboxes = at.tonumpy(_bboxes[0])
    labels = at.tonumpy(_labels[0]).reshape(-1)
    scores = at.tonumpy(_scores[0]).reshape(-1)
        
    person_class_id = 14
    person_indices = labels == person_class_id
    person_bboxes = bboxes[person_indices]
    person_labels = labels[person_indices]
    person_scores = scores[person_indices]
    
    bounding_boxes = person_bboxes
    
    bounding_boxes = [[int(coordinate) for coordinate in box] for box in bounding_boxes]
    
    for i, box in enumerate(bounding_boxes):
        x1, y1, x2, y2 = box
        cropped_image = image[x1:x2, y1:y2]
        for key, value in dict_testg100_imgs.items():
            if img_name in value:
                person_identity = key
                cv2.imwrite(f'datasets/cuhk_sysu/bounding_box_test/{person_identity}_c1s1_000000_0{img_name_cleaner}', cropped_image)
            #else:
                #cv2.imwrite(f'datasets/cuhk_sysu/bounding_box_test/00000_c1s1_000000_0{img_name_cleaner}', cropped_image)
                #break

    #print(str(sample_img[1][0][1][0]))
    #print(str(sample_img[1][0][1][1]))
    #print(str(sample_img[1]))
    #print("{}, {}".format(str(sample_img[0]), str(sample_img[1])))

#lst_person_cleaned = [s.replace('[', '').replace(']', '').replace("'", '').replace('"', '') for s in lst_person]

#unique_set = set(lst_person_cleaned)

# Check for duplicates
#if len(unique_set) != len(lst_person_cleaned):
    #print("There are duplicate elements.")
#else:
    #print("All elements are unique.")

#print(sorted(lst_person_cleaned))
#print(i)

In [ ]:
# Load MAT file
mat = scipy.io.loadmat('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/annotation/test/train_test/0001_c1s1_001051_00_good.mat')

# # Display all variable names contained in the file
print(mat.keys())

# Print header and version information
#print("Header:", mat['__header__'])
#print("Version:", mat['__version__'])
#print("Globals:", mat['__globals__'])

# Access and print data in 'TestG50'
test_data = mat['good_index']
print("Test Data:", test_data)

In [ ]:
# Load MAT file
mat = scipy.io.loadmat('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/annotation/Images.mat')

# # Display all variable names contained in the file
#print(mat.keys())

# Print header and version information
print("Header:", mat['__header__'])
print("Version:", mat['__version__'])
print("Globals:", mat['__globals__'])

# Access and print data in 'Person'
image_data = mat['Img']
#print("Image Data:", image_data)

# Since 'Image' is likely a structured array in MATLAB, it's loaded as a numpy structured array in Python.
# Let's iterate through each person and print their data in a more structured way.

# Loop through each image entry in the array
for image in image_data[0]:  # Assuming image_data[0] to get to the actual data if it's structured with an extra dimension
    image_name = image[0][0]  # Image name
    detection_count = image[1][0]  # Number of detections
    print(f"Image Name: {image_name}")
    print(f"Detections: {detection_count}")
    print("Details:")
    
    # Each 'detail' contains 'idlocate' and 'ishard'
    for detail in image[2]:  
        # Coordinates are in 'idlocate' and the status (e.g., is hard) in 'ishard'
        coordinates = detail['idlocate'][0]
        is_hard = detail['ishard'][0][0]
        print(f"  Coordinates: {coordinates}")
        print(f"  Is Hard: {is_hard}")
        print("-" * 30)

    print("=" * 60)

In [ ]:
# Load MAT file
mat = scipy.io.loadmat('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/annotation/Person.mat')

# Print header and version information
print("Header:", mat['__header__'])
print("Version:", mat['__version__'])
print("Globals:", mat['__globals__'])

# Access and print data in 'Person'
#person_data = mat['Person']
#print("Person Data:", person_data)

# Since 'Person' is likely a structured array in MATLAB, it's loaded as a numpy structured array in Python.
# Let's iterate through each person and print their data in a more structured way.

# Assuming person_data is a structured array with fields that can be indexed by name
for person in person_data[0]:  # person_data[0] because it is likely a 2D array with one actual dimension
    print("Person ID:", person[0][0])  # Person ID
    print("Appearing Time:", person[1][0])  # Some count, assuming it's in the second position
    print("Scene Location:")
    for detail in person[2]:  # Assuming details are in the third position and it's an array of structured arrays
        print(f"  Image Name: {detail['imname'][0]}")
        print(f"  Location: {detail['idlocate'][0]}")
        print(f"  Unknown: {detail['ishard'][0][0]}")
        print("-" * 30)
    print("=" * 30)

In [ ]:
# Load MAT file
mat = scipy.io.loadmat('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/annotation/pool.mat')

# # Display all variable names contained in the file
print(mat.keys())

# Print header and version information
print("Header:", mat['__header__'])
print("Version:", mat['__version__'])
print("Globals:", mat['__globals__'])

# Access and print data in 'pool'
pool_data = mat['pool']
print("Pool Data:", pool_data)

In [ ]:
# Load MAT file
mat = scipy.io.loadmat('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/annotation/test/subset/Occlusion.mat')

# # Display all variable names contained in the file
print(mat.keys())

# Print header and version information
print("Header:", mat['__header__'])
print("Version:", mat['__version__'])
print("Globals:", mat['__globals__'])

# Access and print data in 'pool'
pool_data = mat['Occlusion1']
print("Test_Size Data:", pool_data)

# Access 'Test_Size' data
#test_size_data = mat['Test_Size']

# Print only the first few elements of 'Test_Size' to avoid too much output
#print("Test_Size Data (first few entries):", test_size_data[:5])

# Additional summary information, like the shape and datatype
#print("Total number of entries in Test_Size:", test_size_data.size)
#print("Data type of Test_Size entries:", test_size_data.dtype)